# Busca de Produtos no Mercado Livre e Criação de Dataset

Este notebook tem como objetivo buscar informações de produtos no Mercado Livre utilizando a API pública, 
processar esses dados e criar um dataset (arquivo CSV) com as informações relevantes.

In [ ]:
import requests
import pandas as pd

## Configurações

Define as variáveis de configuração, como token de acesso, URLs da API e termos de pesquisa.

In [ ]:
ACCESS_TOKEN = 'APP_USR-SEU_TOKEN'
BASE_URL_SEARCH = 'https://api.mercadolibre.com/sites/MLB/search?q='
BASE_URL_ITEM = 'https://api.mercadolibre.com/items/'
HEADERS = {'Authorization': f'Bearer {ACCESS_TOKEN}'}
SEARCH_TERMS = [
    'Google Chromecast',
    'Amazon Fire TV Stick',
    'Roku Streaming Stick',
    'Xiaomi Mi Box S',
    'Apple TV 4K'
]

## Função para buscar resultados de pesquisa

Esta função busca os resultados da pesquisa para um determinado termo, utilizando a API de busca do Mercado Livre.

In [3]:
def fetch_search_results(search_term):
    offset = 0
    total_results = 0
    search_results = []

    while offset <= total_results:
        url = f"{BASE_URL_SEARCH}{search_term.lower().replace(' ', '%20')}&offset={offset}"
        response = requests.get(url, headers=HEADERS)
        data = response.json()
        total_results = data['paging']['total']
        limit = data['paging']['limit']
        offset += limit
        search_results.append(data)
    return search_results

## Função para extrair IDs dos itens

Esta função extrai os IDs dos itens dos resultados da pesquisa.

In [4]:
def extract_item_ids(search_results):
    item_ids = []
    for result in search_results:
        for item in result['results']:
            item_ids.append(item['id'])
    return item_ids

## Função para buscar detalhes dos itens

Esta função busca os detalhes dos itens utilizando seus IDs, através da API de itens do Mercado Livre.


In [5]:
def fetch_item_details(item_ids):
    item_details = []
    for item_id in item_ids:
        url = f"{BASE_URL_ITEM}{item_id}"
        response = requests.get(url, headers=HEADERS)
        item_details.append(response.json())
    return item_details

## Função para obter todas as chaves de atributos

Esta função obtém todas as chaves de atributos possíveis dos detalhes dos itens, para padronização.


In [6]:
def get_all_attribute_keys(item_details):
    all_attributes = set()
    for item in item_details:
        if "attributes" in item:
            for attribute in item["attributes"]:
                attribute_id = attribute.get("id")
                if attribute_id:
                    all_attributes.add(attribute_id.lower())
    return all_attributes

## Função para achatar os detalhes dos itens

Esta função achata os detalhes dos itens, movendo os atributos para o nível superior e garantindo que todas as chaves de atributo possíveis estejam presentes.

In [7]:
def flatten_item_details(item_details, all_attributes):
    flattened_details = []
    for item in item_details:
        flattened_item = item.copy()
        if "attributes" in flattened_item:
            for attribute in flattened_item["attributes"]:
                attribute_id = attribute.get("id")
                attribute_value = attribute.get("value_name")
                if attribute_id:
                    flattened_item[attribute_id.lower()] = attribute_value

        for attribute_key in all_attributes:
            if attribute_key not in flattened_item:
                flattened_item[attribute_key] = None
        flattened_details.append(flattened_item)
    return flattened_details

## Função para selecionar e ordenar colunas

Esta função seleciona e ordena as colunas dos detalhes dos itens achatados.

In [8]:
def select_and_order_columns(flattened_details, selected_columns):
    final_details = []
    for item in flattened_details:
        final_item = {k: item[k] for k in item if k in selected_columns}
        final_details.append(final_item)
    return final_details

## Lista de colunas selecionadas

Define a lista de colunas que serão incluídas no dataset final.

In [9]:
SELECTED_COLUMNS = [
    "id", "site_id", "title", "seller_id", "category_id",
    "official_store_id", "price", "base_price", "original_price",
    "currency_id", "initial_quantity", "buying_mode", "listing_type_id",
    "condition", "permalink", "accepts_mercadopago", "shipping_mode",
    "free_shipping", "shipping_logistic_type", "international_delivery_mode",
    "status", "warranty", "date_created", "last_updated",
    "alphanumeric_model", "anatel_homologation_number", "audio_and_video_interfaces",
    "audio_quality", "brand", "color", "compatible_applications", "cpu",
    "device_format", "device_operating_voltage", "frequencies", "generation",
    "graphics_processor", "height", "includes_hdmi_cable",
    "integrated_voice_assistants", "is_kit", "item_condition", "language_options",
    "length", "main_color", "manufacturer", "max_video_resolution",
    "min_operating_systems_required", "model", "mpn", "os_name", "package_height",
    "package_length", "package_weight", "package_width", "ports_and_connectors_types",
    "product_features", "recommended_devices", "seller_sku", "shipment_packing",
    "streaming_technologies", "weight", "width", "with_bluetooth",
    "with_integrated_voice_assistants", "with_wi_fi", "wi_fi_standards",
    "is_dust_resistant", "ram_memory", 
    "product_data_source", "includes_charger"
]

## Execução e Processamento dos Dados

Este bloco executa todas as etapas de busca, processamento e criação do dataset.

In [10]:
all_search_results = []
for term in SEARCH_TERMS:
    all_search_results.extend(fetch_search_results(term))

all_item_ids = extract_item_ids(all_search_results)
all_item_details = fetch_item_details(all_item_ids)
all_attribute_keys = get_all_attribute_keys(all_item_details)
flattened_item_details = flatten_item_details(all_item_details, all_attribute_keys)
final_item_details = select_and_order_columns(flattened_item_details, SELECTED_COLUMNS)
df_final = pd.DataFrame(final_item_details)
df_final.to_csv('dataset.csv', index=False)